<h2 style='text-align: center;'>RAG: Overview</h2>

<p align='center'>
  <img src='./rag.png' width=480 style='border-radius: 14px;' />
</p>

<p align='justify' style='padding-left: 20rem; padding-right: 20rem; padding-top: 2rem;'>
  &nbsp;&nbsp;&nbsp;Neste notebook, pretendo mergulhar em um guia rápido para um projeto RAG usando LangChain. O projeto LangChain é uma plataforma baseada em blockchain que tem como objetivo fornecer uma maneira descentralizada e segura de armazenar e compartilhar dados linguísticos. A plataforma é construída em cima da blockchain Ethereum e usa o protocolo IPFS para armazenar os dados. A plataforma é projetada para ser usada por pesquisadores de linguagem, linguistas e entusiastas de linguagem que desejam armazenar e compartilhar dados linguísticos de forma segura e descentralizada.
</p>

<br>

<h3>Setting Up</h3>

In [4]:
import sys
sys.path.append('D:/Deep Learning/Diver')

from config.constants import *
from config.config import *
from config.utils import *

warnings.filterwarnings('ignore')

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = f'RAG'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = settings.LANGSMITH_API_KEY

<h3>Indexing</h3>

In [5]:
hf_embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

loader = WebBaseLoader(
  web_paths=('https://github.com/naomilago?tab=repositories',)
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
splits = text_splitter.split_documents(docs)

vector_store = Chroma.from_documents(documents=splits, embedding=hf_embeddings)

<h3>Retrieval and Generation</h3>

In [6]:
retriever = vector_store.as_retriever()

template = '''
You are a reference in academic communication, focusing on all kind of target audience. 
Everyone listen and understand what you have to say, so you always answer with a great markdown syntax and in a very assertive, clear, and concise language. Use your knowledge to answer the following:

{context}

{question}
'''

prompt = PromptTemplate(template=template, input_variables=['question'])

llm = ChatGoogleGenerativeAI(
  google_api_key=settings.GEMINI_API_KEY,
  model='gemini-1.5-pro-latest',
  temperature=0
)

docs_formatter = lambda docs: str.join('\n\n', (_.page_content for _ in docs))

chain = RunnableSequence((
  {'context': retriever | docs_formatter, 'question': RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
))

result = chain.invoke('Quais os repositorios mais populares de Naomi Lago?')
display(Markdown(result))

## Repositórios Mais Populares de Naomi Lago

Analisando o perfil do GitHub de Naomi Lago, podemos destacar os seguintes repositórios como os mais populares, considerando a quantidade de estrelas e a descrição dos projetos:

*   **diver:** Este repositório abriga o código do Diver, um bot com inteligência artificial projetado para auxiliar usuários na exploração do mundo da Ciência de Dados. Ele oferece conhecimento técnico e insights do blog pessoal de Naomi.
*   **fitness-tracker:** Este projeto envolve o processamento, análise e modelagem de dados de condicionamento físico para classificar exercícios com barra e contar repetições.
*   **Falconize:** Um projeto Capstone de Ciência de Dados Aplicada oferecido pela IBM na Coursera.
*   **Mentoria\_Python:** Este repositório compartilha notebooks para mentorias em programação Python.
*   **detecting-programming-lanugages:** Um projeto para praticar o uso do SpaCy para Reconhecimento de Entidades Nomeadas (NER) em linguagens de programação.

Esses repositórios demonstram a expertise de Naomi Lago em Ciência de Dados e áreas relacionadas, como Processamento de Linguagem Natural (PNL) e aprendizado de máquina. 
